In [1]:
!pip install -q bitsandbytes accelerate transformers peft datasets trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset

dataset = load_dataset("tatsu-lab/alpaca")

# Take only a small sample if you're on free Colab
dataset = dataset["train"].select(range(500))


def format_prompt(example):
    return {
        "input_ids": tokenizer(
            f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}",
            truncation=True,
            padding="max_length",
            max_length=512
        )["input_ids"]
    }

tokenized_dataset = dataset.map(format_prompt)



README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [4]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)


In [5]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./tinyllama-alpaca-lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=2,
    save_strategy="epoch",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [6]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.648500
20,1.272500
30,1.244100
40,1.319500
50,1.190200
60,1.256500
70,1.191500
80,1.135400
90,1.144200
100,1.073800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=189, training_loss=1.1804200682059798, metrics={'train_runtime': 633.4725, 'train_samples_per_second': 2.368, 'train_steps_per_second': 0.298, 'total_flos': 4787794870272000.0, 'train_loss': 1.1804200682059798, 'epoch': 3.0})

In [7]:
model.save_pretrained("tinyllama-finetuned-alpaca")
tokenizer.save_pretrained("tinyllama-finetuned-alpaca")


('tinyllama-finetuned-alpaca/tokenizer_config.json',
 'tinyllama-finetuned-alpaca/special_tokens_map.json',
 'tinyllama-finetuned-alpaca/chat_template.jinja',
 'tinyllama-finetuned-alpaca/tokenizer.model',
 'tinyllama-finetuned-alpaca/added_tokens.json',
 'tinyllama-finetuned-alpaca/tokenizer.json')

In [9]:
!huggingface-cli login  # Login using token from your account


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `google collab` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /r

In [10]:
from huggingface_hub import create_repo, HfApi

repo_name = "tinyllama-alpaca-finetuned"
create_repo(repo_name, private=True)  # Or private=False for public repo
trainer.push_to_hub(repo_name)


adapter_model.safetensors:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aswanth6001/tinyllama-alpaca-lora/commit/36d4f5d448bbb5b449cb251fd66ed65e6f3241f3', commit_message='tinyllama-alpaca-finetuned', commit_description='', oid='36d4f5d448bbb5b449cb251fd66ed65e6f3241f3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/aswanth6001/tinyllama-alpaca-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='aswanth6001/tinyllama-alpaca-lora'), pr_revision=None, pr_num=None)

In [15]:
model.push_to_hub("tinyllama-alpaca-finetuned")
tokenizer.push_to_hub("tinyllama-alpaca-finetuned")


adapter_model.safetensors:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aswanth6001/tinyllama-alpaca-finetuned/commit/2c48873c6e3a1e002cfe613143ae3ed65998e59b', commit_message='Upload tokenizer', commit_description='', oid='2c48873c6e3a1e002cfe613143ae3ed65998e59b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/aswanth6001/tinyllama-alpaca-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='aswanth6001/tinyllama-alpaca-finetuned'), pr_revision=None, pr_num=None)

In [16]:
from transformers import pipeline

pipe = pipeline("text-generation", model="aswanth6001/tinyllama-alpaca-finetuned", tokenizer="aswanth6001/tinyllama-alpaca-finetuned")

prompt = "### Instruction:\nExplain photosynthesis.\n\n### Response:\n"
result = pipe(prompt, max_new_tokens=100)
print(result[0]["generated_text"])


adapter_config.json:   0%|          | 0.00/868 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.0M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/951 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/410 [00:00<?, ?B/s]

Device set to use cuda:0


### Instruction:
Explain photosynthesis.

### Response:
Photosynthesis is the process by which plants and other organisms convert sunlight into energy and oxygen. It is the primary process by which organisms obtain energy from the sun. Plants and other organisms use carbon dioxide and water to combine to produce sugars and oxygen. The energy used in photosynthesis is stored in the form of stored carbohydrates, which are used by the organism for growth and other functions. The process of photosyn
